In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [20]:
data = yf.download('^NSEBANK', '2015-01-01', '2024-12-15')

# 3. Data Preparation
data = pd.DataFrame(data)
data.reset_index(inplace=True)

# Check for missing values
if data.isnull().values.any():
    data.fillna(method='ffill', inplace=True)  


data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data = data[['Close']]  # Keep only the 'Close' column

C:\Users\Aayush\AppData\Local\Temp\ipykernel_11484\3249301671.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('^NSEBANK', '2015-01-01', '2024-12-15')
[*********************100%***********************]  1 of 1 completed


In [21]:
print(data)

             Close
0     18751.982055
1     19154.976916
2     18874.379592
3     18382.336980
4     18586.884138
...            ...
2169  53380.750000
2170  53450.050781
2171  53459.300781
2172  53201.000000
2173  53109.800781

[2174 rows x 1 columns]


In [22]:
train_data = data[:-50]
test_data = data[-50:]

In [23]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)

# Scale the test data using the same scaler
test_data_scaled = scaler.transform(test_data)


In [24]:
base_days = 10
x_train, y_train = [], []
for i in range(base_days, train_data_scaled.shape[0]):
    x_train.append(train_data_scaled[i-base_days:i])
    y_train.append(train_data_scaled[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [25]:
model = Sequential()
model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(60, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(80, activation='tanh', return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(120, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=1))

c:\Users\Aayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
model.fit(x_train, y_train, epochs=50, batch_size=32, callbacks=[early_stopping])

Epoch 1/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0705
Epoch 2/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0052
Epoch 3/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0048
Epoch 4/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0045
Epoch 5/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0035
Epoch 6/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0042
Epoch 7/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0038
Epoch 8/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0036
Epoch 9/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0032
Epoch 10/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0038
Epoch 11/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0030
Epoch 12/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0029
Epoch 13/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0050
Epoch 14/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0032
Epoch 15/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0028
Epoch 1

In [28]:
x_test, y_test = [], []
for i in range(base_days, test_data_scaled.shape[0]):
    x_test.append(test_data_scaled[i-base_days:i])
    y_test.append(test_data_scaled[i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Predicting
predictions = model.predict(x_test)
predictions = predictions.reshape(-1, 1)

# Inverse transform to get actual price values
predictions = scaler.inverse_transform(predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 345ms/step


In [29]:
future_days = 10
last_sequence = x_train[-1:]  # This takes the last sequence from your training data
future_predictions = []

for i in range(future_days):
    pred = model.predict(last_sequence, verbose=0)
    future_predictions.append(pred[0][0])
    
    # Update last_sequence
    new_sequence = last_sequence[0]
    new_sequence = np.roll(new_sequence, -1)
    new_sequence[-1] = pred[0][0]
    last_sequence = new_sequence.reshape(1, base_days, 1)

In [30]:
future_predictions = np.array(future_predictions).reshape(-1, 1)
future_predictions = scaler.inverse_transform(future_predictions)

In [31]:
future_predictions

array([[54103.344],
       [54307.66 ],
       [54493.793],
       [54612.168],
       [54708.64 ],
       [54768.32 ],
       [54833.87 ],
       [54936.48 ],
       [55030.94 ],
       [55115.984]], dtype=float32)

In [32]:
model.save('Niftystockmodel.keras')